# Model Nasha - Chwilowy Hydrogram Jednostkowy

Ten notebook przedstawia model Nasha (Nash IUH) i metody estymacji jego parametrów.

## Teoria

Model Nasha reprezentuje zlewnię jako **kaskadę n identycznych zbiorników liniowych**, każdy ze stałą retencji K:

```
Opad → [Zbiornik 1] → [Zbiornik 2] → ... → [Zbiornik n] → Odpływ
           K              K                    K
```

**Wzór IUH (Chwilowego Hydrogramu Jednostkowego):**

$$u(t) = \frac{1}{K \cdot \Gamma(n)} \cdot \left(\frac{t}{K}\right)^{n-1} \cdot e^{-t/K}$$

**Właściwości:**
- Czas opóźnienia (lag): $t_{lag} = n \cdot K$
- Czas do szczytu: $t_p = (n-1) \cdot K$ (dla n > 1)
- Wariancja: $\sigma^2 = n \cdot K^2$

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))

# Import bibliotek
from hydrolog.runoff import NashIUH, HydrographGenerator
from hydrolog.precipitation import BetaHietogram
from hydrolog.time import ConcentrationTime
import numpy as np

## 1. Podstawowe użycie modelu Nasha

Najprostszy sposób - bezpośrednie podanie parametrów n i K.

In [ ]:
# Parametry modelu
n = 3.0      # liczba zbiorników (parametr kształtu)
k_min = 30.0 # stała retencji [min]

# Utworzenie modelu (bez powierzchni - zwraca IUH bezwymiarowy)
iuh = NashIUH(n=n, k_min=k_min)

print(f"Parametry modelu Nasha:")
print(f"  n = {iuh.n}")
print(f"  K = {iuh.k_min} min")
print(f"")
print(f"Charakterystyki IUH:")
print(f"  Czas opóźnienia (tlag): {iuh.lag_time_min:.1f} min")
print(f"  Czas do szczytu (tp): {iuh.time_to_peak_min:.1f} min")
print(f"  Ordynata szczytowa (up): {iuh.peak_ordinate_per_min:.6f} 1/min")

In [ ]:
# Generowanie IUH
result_iuh = iuh.generate_iuh(timestep_min=5.0, duration_min=300.0)

print(f"IUH wygenerowany:")
print(f"  Liczba kroków: {result_iuh.n_steps}")
print(f"  Suma ordinat × dt ≈ {np.trapz(result_iuh.ordinates_per_min, result_iuh.times_min):.4f} (powinno ≈ 1.0)")

## 2. Model z powierzchnią zlewni (wymiarowy UH)

Gdy podamy powierzchnię zlewni, model zwraca hydrogram jednostkowy w jednostkach [m³/s na mm opadu].

In [ ]:
# Model z powierzchnią zlewni
area_km2 = 45.0

nash = NashIUH(n=3.0, k_min=30.0, area_km2=area_km2)
result_uh = nash.generate(timestep_min=5.0)

print(f"Hydrogram jednostkowy (UH):")
print(f"  Powierzchnia zlewni: {result_uh.area_km2} km²")
print(f"  Czas trwania opadu (D): {result_uh.duration_min} min")
print(f"  Qmax: {result_uh.peak_discharge_m3s:.3f} m³/s na mm")
print(f"  Czas do szczytu: {result_uh.time_to_peak_min:.1f} min")

# Weryfikacja objętości (1 mm na A km² = A × 1000 m³)
volume_m3 = np.trapz(result_uh.ordinates_m3s, result_uh.times_min * 60)  # t w sekundach
expected_volume = area_km2 * 1000
print(f"")
print(f"Weryfikacja objętości:")
print(f"  Obliczona: {volume_m3:,.0f} m³")
print(f"  Oczekiwana: {expected_volume:,.0f} m³")
print(f"  Błąd: {abs(volume_m3-expected_volume)/expected_volume*100:.2f}%")

## 3. Metody estymacji parametrów

### 3.1 Z czasu koncentracji (`from_tc`)

Najprostsza metoda - wymaga założenia wartości n.

In [ ]:
# Oblicz czas koncentracji
tc = ConcentrationTime.kirpich(length_km=8.2, slope_m_per_m=0.023)
print(f"Czas koncentracji (Kirpich): {tc:.1f} min")

# Estymacja parametrów z tc
# tlag = 0.6 × tc (relacja SCS)
# K = tlag / n
nash_tc = NashIUH.from_tc(tc_min=tc, n=3.0, lag_ratio=0.6)

print(f"")
print(f"Estymacja z tc (n=3.0, lag_ratio=0.6):")
print(f"  tlag = 0.6 × {tc:.1f} = {0.6*tc:.1f} min")
print(f"  K = {nash_tc.k_min:.2f} min")
print(f"  Weryfikacja: tlag = n × K = {nash_tc.lag_time_min:.1f} min")

### 3.2 Z momentów statystycznych (`from_moments`)

Gdy mamy obserwowany hydrogram, możemy obliczyć momenty statystyczne.

In [ ]:
# Dla modelu Nasha:
# M1 = tlag = n × K (pierwszy moment - średnia)
# M2 = σ² = n × K² (drugi moment - wariancja)
#
# Rozwiązując:
# K = M2 / M1
# n = M1² / M2

# Przykład: załóżmy że z obserwacji mamy:
lag_time_obs = 90.0      # tlag = 90 min
variance_obs = 2700.0    # σ² = 2700 min²

nash_moments = NashIUH.from_moments(lag_time_min=lag_time_obs, variance_min2=variance_obs)

print(f"Estymacja z momentów:")
print(f"  Dane: tlag = {lag_time_obs} min, σ² = {variance_obs} min²")
print(f"  K = σ²/tlag = {variance_obs}/{lag_time_obs} = {nash_moments.k_min:.1f} min")
print(f"  n = tlag²/σ² = {lag_time_obs}²/{variance_obs} = {nash_moments.n:.2f}")

### 3.3 Metoda Lutza (`from_lutz`)

**Metoda Lutza** służy do estymacji parametrów modelu Nasha z charakterystyk fizjograficznych zlewni. Zalecana dla **zlewni niezurbanizowanych** bez danych pomiarowych.

**Algorytm:**
1. $P_1 = 3.989 \cdot n_{Manning} + 0.028$
2. $t_p = P_1 \cdot \left(\frac{L \cdot L_c}{J_g^{1.5}}\right)^{0.26} \cdot e^{-0.016 U} \cdot e^{0.004 W}$ [h]
3. $u_p = \frac{0.66}{t_p^{1.04}}$ [1/h]
4. $f(N) = t_p \cdot u_p$ → N (z tabeli lub numerycznie)
5. $K = \frac{t_p}{N-1}$ [h]

gdzie:
- L - długość cieku głównego [km]
- Lc - długość do centroidu zlewni [km]
- Jg - spadek cieku [-]
- n - współczynnik Manninga [-]
- U - udział urbanizacji [%]
- W - udział lasów [%]

In [ ]:
# Parametry zlewni
L_km = 15.0          # Długość cieku głównego [km]
Lc_km = 8.0          # Długość do centroidu zlewni [km]
slope = 0.02         # Spadek cieku [-] (2%)
manning_n = 0.035    # Współczynnik Manninga [-]
forest_pct = 40.0    # Udział lasów [%]
urban_pct = 5.0      # Udział urbanizacji [%]
area_km2 = 50.0      # Powierzchnia zlewni [km²]

# Estymacja metodą Lutza
nash_lutz = NashIUH.from_lutz(
    L_km=L_km,
    Lc_km=Lc_km,
    slope=slope,
    manning_n=manning_n,
    forest_pct=forest_pct,
    urban_pct=urban_pct,
    area_km2=area_km2
)

print(f"Metoda Lutza - estymacja parametrów:")
print(f"")
print(f"Dane wejściowe:")
print(f"  L = {L_km} km, Lc = {Lc_km} km")
print(f"  Spadek = {slope*100:.1f}%")
print(f"  Manning n = {manning_n}")
print(f"  Las = {forest_pct}%, Urbanizacja = {urban_pct}%")
print(f"")
print(f"Wyniki:")
print(f"  N = {nash_lutz.n:.3f}")
print(f"  K = {nash_lutz.k_min:.1f} min ({nash_lutz.k_min/60:.3f} h)")
print(f"  tlag = {nash_lutz.lag_time_min:.1f} min")
print(f"  tp = {nash_lutz.time_to_peak_min:.1f} min")

### Wpływ parametrów fizjograficznych

In [ ]:
# Porównanie: wpływ lesistości
print("Wpływ lesistości na parametry modelu:")
print("-" * 60)
print(f"{'Las [%]':<10} {'N':>10} {'K [min]':>12} {'tlag [min]':>12}")
print("-" * 60)

for forest in [0, 20, 40, 60, 80]:
    nash = NashIUH.from_lutz(
        L_km=15.0, Lc_km=8.0, slope=0.02, manning_n=0.035,
        forest_pct=forest, urban_pct=0.0
    )
    print(f"{forest:<10} {nash.n:>10.3f} {nash.k_min:>12.1f} {nash.lag_time_min:>12.1f}")

In [ ]:
# Porównanie: wpływ urbanizacji
print("Wpływ urbanizacji na parametry modelu:")
print("-" * 60)
print(f"{'Urban [%]':<10} {'N':>10} {'K [min]':>12} {'tlag [min]':>12}")
print("-" * 60)

for urban in [0, 10, 20, 30, 40]:
    nash = NashIUH.from_lutz(
        L_km=15.0, Lc_km=8.0, slope=0.02, manning_n=0.035,
        forest_pct=30.0, urban_pct=urban
    )
    print(f"{urban:<10} {nash.n:>10.3f} {nash.k_min:>12.1f} {nash.lag_time_min:>12.1f}")

## 4. Generowanie hydrogramu odpływu

Model Nasha można użyć w `HydrographGenerator` do transformacji opadu w odpływ.

In [ ]:
# Parametry zlewni
area_km2 = 50.0
cn = 72

# Hietogram opadu
hietogram = BetaHietogram(alpha=2.0, beta=5.0)
precip = hietogram.generate(total_mm=50.0, duration_min=120.0, timestep_min=10.0)

# Generator z modelem Nasha (estymacja Lutza)
generator = HydrographGenerator(
    area_km2=area_km2,
    cn=cn,
    uh_model="nash",
    uh_params={
        "n": nash_lutz.n,
        "k": nash_lutz.k_min / 60,  # k w godzinach!
        "k_unit": "hours"
    }
)

result = generator.generate(precip)

print(f"Hydrogram odpływu (model Nasha):")
print(f"="*50)
print(f"Parametry Nasha: n={nash_lutz.n:.3f}, K={nash_lutz.k_min:.1f} min")
print(f"")
print(f"Opad całkowity: {result.total_precip_mm:.1f} mm")
print(f"Opad efektywny: {result.total_effective_mm:.1f} mm")
print(f"Współczynnik odpływu: {result.runoff_coefficient:.3f}")
print(f"")
print(f"Przepływ maksymalny: {result.peak_discharge_m3s:.2f} m³/s")
print(f"Czas do szczytu: {result.time_to_peak_min:.0f} min")
print(f"Objętość odpływu: {result.total_volume_m3:,.0f} m³")

## 5. Porównanie modeli UH

In [ ]:
# Porównanie: SCS vs Nash
tc = ConcentrationTime.kirpich(length_km=15.0, slope_m_per_m=0.02)

# Model SCS
gen_scs = HydrographGenerator(area_km2=50.0, cn=72, tc_min=tc)
result_scs = gen_scs.generate(precip)

# Model Nash (z tc)
nash_tc = NashIUH.from_tc(tc_min=tc, n=3.0)
gen_nash = HydrographGenerator(
    area_km2=50.0, cn=72,
    uh_model="nash",
    uh_params={"n": nash_tc.n, "k": nash_tc.k_min/60, "k_unit": "hours"}
)
result_nash = gen_nash.generate(precip)

# Model Nash (z Lutza)
gen_lutz = HydrographGenerator(
    area_km2=50.0, cn=72,
    uh_model="nash",
    uh_params={"n": nash_lutz.n, "k": nash_lutz.k_min/60, "k_unit": "hours"}
)
result_lutz = gen_lutz.generate(precip)

print("Porównanie modeli UH:")
print("="*70)
print(f"{'Model':<20} {'Qmax [m³/s]':>15} {'tp [min]':>12} {'V [m³]':>15}")
print("-"*70)
print(f"{'SCS':<20} {result_scs.peak_discharge_m3s:>15.2f} {result_scs.time_to_peak_min:>12.0f} {result_scs.total_volume_m3:>15,.0f}")
print(f"{'Nash (z tc)':<20} {result_nash.peak_discharge_m3s:>15.2f} {result_nash.time_to_peak_min:>12.0f} {result_nash.total_volume_m3:>15,.0f}")
print(f"{'Nash (Lutz)':<20} {result_lutz.peak_discharge_m3s:>15.2f} {result_lutz.time_to_peak_min:>12.0f} {result_lutz.total_volume_m3:>15,.0f}")

## 6. Wizualizacja

In [ ]:
# Opcjonalna wizualizacja (wymaga matplotlib)
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 1. IUH dla różnych n
    ax1 = axes[0, 0]
    for n in [2, 3, 4, 5]:
        iuh = NashIUH(n=n, k_min=30.0)
        result = iuh.generate_iuh(timestep_min=2.0, duration_min=300.0)
        ax1.plot(result.times_min, result.ordinates_per_min, label=f'n={n}')
    ax1.set_xlabel('Czas [min]')
    ax1.set_ylabel('u(t) [1/min]')
    ax1.set_title('IUH dla różnych n (K=30 min)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. IUH dla różnych K
    ax2 = axes[0, 1]
    for k in [20, 30, 40, 50]:
        iuh = NashIUH(n=3.0, k_min=k)
        result = iuh.generate_iuh(timestep_min=2.0, duration_min=400.0)
        ax2.plot(result.times_min, result.ordinates_per_min, label=f'K={k} min')
    ax2.set_xlabel('Czas [min]')
    ax2.set_ylabel('u(t) [1/min]')
    ax2.set_title('IUH dla różnych K (n=3)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Wpływ lesistości na UH
    ax3 = axes[1, 0]
    for forest in [0, 30, 60]:
        nash = NashIUH.from_lutz(
            L_km=15.0, Lc_km=8.0, slope=0.02, manning_n=0.035,
            forest_pct=forest, urban_pct=0.0, area_km2=50.0
        )
        result = nash.generate(timestep_min=5.0)
        ax3.plot(result.times_min, result.ordinates_m3s, label=f'Las={forest}%')
    ax3.set_xlabel('Czas [min]')
    ax3.set_ylabel('Q [m³/s/mm]')
    ax3.set_title('Wpływ lesistości na UH (Lutz)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Porównanie modeli
    ax4 = axes[1, 1]
    ax4.plot(result_scs.hydrograph.times_min, result_scs.hydrograph.discharge_m3s, 
             'b-', linewidth=2, label='SCS')
    ax4.plot(result_nash.hydrograph.times_min, result_nash.hydrograph.discharge_m3s, 
             'r--', linewidth=2, label='Nash (z tc)')
    ax4.plot(result_lutz.hydrograph.times_min, result_lutz.hydrograph.discharge_m3s, 
             'g-.', linewidth=2, label='Nash (Lutz)')
    ax4.set_xlabel('Czas [min]')
    ax4.set_ylabel('Q [m³/s]')
    ax4.set_title('Porównanie hydrogramów odpływu')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Matplotlib nie jest zainstalowany. Zainstaluj: pip install matplotlib")

## Podsumowanie

W tym notebooku pokazano:

1. **Teorię modelu Nasha** - kaskada zbiorników liniowych
2. **Metody estymacji parametrów:**
   - `from_tc()` - z czasu koncentracji
   - `from_moments()` - z momentów statystycznych
   - `from_lutz()` - z charakterystyk fizjograficznych
3. **Generowanie IUH i UH** - bezwymiarowe i wymiarowe
4. **Użycie z HydrographGenerator** - transformacja opad-odpływ
5. **Porównanie modeli** - SCS vs Nash

### Kiedy stosować metodę Lutza?

- Zlewnie **niezurbanizowane** lub słabo zurbanizowane
- Brak danych pomiarowych (hydrogram obserwowany)
- Dostępne dane fizjograficzne (L, Lc, spadek, szorstkość)

### Referencje

- Nash, J.E. (1957). The form of the instantaneous unit hydrograph. IAHS Publication 45(3), 114-121.
- Lutz, W. (1984). Berechnung von Hochwasserabflüssen unter Anwendung von Gebietskenngrößen. Universität Karlsruhe.
- KZGW (2017). Aktualizacja metodyki obliczania przepływów i opadów maksymalnych. Tabela C.2.